In [ ]:
import sys
import os
import importlib as implib

import re

from collections import OrderedDict as odict

import numpy as np
import pandas as pd

from tqdm import tqdm

In [ ]:
import autods as ads

# Tests unitaires module mcds.

## 1. Classe DataSet

In [ ]:
dfData = pd.DataFrame(columns=['Date', 'TrucDec', 'Espece', 'Point', 'Effort', 'Distance'],
                      data=[('2019-05-13', 3.5, 'TURMER', 23, 2,   83),
                            ('2019-05-15', np.nan, 'TURMER', 23, 2,   27.355),
                            ('2019-05-13', 0, 'ALAARV', 29, 2,   56.85),
                            ('2019-04-03', 1.325, 'PRUMOD', 53, 1.3,  7.2),
                            ('2019-06-01', 2, 'PHICOL', 12, 1,  np.nan),
                            ('2019-06-19', np.nan, 'PHICOL', 17, np.nan, np.nan),
                           ])
dfData['Region'] = 'ACDC'
dfData['Surface'] = '2400'
dfData

In [ ]:
ds = ads.DataSet(dfData, decimalFields=['Effort', 'Distance', 'TrucDec'])

## 2. Classes XXEngine

### a. Instanciation et détection de Distance

In [ ]:
_ = implib.reload(ads)

In [ ]:
# Hack up normal installation root folders, in order to test without really installing Distance
ads.DSEngine.DistancePossInstPaths = ['ads']

In [ ]:
eng = ads.MCDSEngine(workDir='ads')

### b. Génération fichier de données en entrée de MCDS

In [ ]:
_ = eng.setupRunFolder(runName='mcds-test')

In [ ]:
dataFileName = eng.buildDataFile(dataSet=ds)

### c. Génération fichier de "commandes"

In [ ]:
cmdFileName = eng.buildCmdFile(estimKeyFn='HNORMAL', estimAdjustFn='COSINE',
                               estimCriterion='AIC', cvInterval=95)

### d. Génération fichier de données en entrée pour Distance

(mode 'point transect' uniquement pour le moment)

In [ ]:
distDataFileName = \
    eng.buildDistanceDataFile(ds.dfData, tgtFilePathName=os.path.join('ads', 'distance-input-test.txt'),
                              decimalFields=ds.decimalFields)

### d. Dry run (only generate input and cmd files)

In [ ]:
eng(ds, runName='mcds-test', realRun=False,
    estimKeyFn='UNIFORM', estimAdjustFn='POLY',
    estimCriterion='AIC', cvInterval=95)

# Tests de validation module ads

## 1. Génération de fichiers d'entrée pour Distance

* via un jeu de fichiers d'entrée bruts Excel, et leur export de référence, éprouvé dans Distance,
* et comparaison du produit de XXEngine.buildDistanceDataFile à cette référence.

In [ ]:
os.makedirs(os.path.join('AutoDS', 'tmp'), exist_ok=True)

In [ ]:
testCases = [dict(fileName='ACDC2019-Papyrus-ALAARV-Males-AB-10mn-saisie.xlsx',
                  decimalFields=['EFFORT', 'DISTANCE', 'NOMBRE'])]

In [ ]:
# Hack up normal installation root folders, in order to test without really installing Distance
ads.DSEngine.DistancePossInstPaths = ['AutoDS']

In [ ]:
eng = ads.MCDSEngine(workDir='AutoDS')

In [ ]:
for ind, tc in enumerate(testCases):
    
    fn = tc['fileName']
    print('#', ind, ':', fn)
    dfData = pd.read_excel(os.path.join('AutoDS', 'test-ref', fn))
    rfn = eng.buildDistanceDataFile(dfData, os.path.join('AutoDS', 'tmp', os.path.splitext(fn)[0]+'.txt'),
                                    decimalFields=tc['decimalFields'])
    print()
    
    #TODO: Comparer à référence ?

## 2. Analyses massives ACDC Papier 2019

In [ ]:
def extraireJeuDonnees(dfTout, espece, passages=['A', 'B'], duree='10mn'):
    
    assert all(p in ['A', 'B'] for p in passages)
    assert duree in ['5mn', '10mn']
    assert espece in dfTout.ESPECE.unique()
    
    # Passages
    dfJeu = dfTout[(dfTout.ESPECE == espece) & (dfTout.PASSAGE.isin(passages))].copy()
    
    # Durée
    if duree == '10mn':
        dfJeu['NOMBRE'] = dfJeu[['PER5MN', 'PER10MN']].sum(axis='columns')
    else:
        dfJeu['NOMBRE'] = dfJeu['PER5MN']
    dfJeu.drop(dfJeu[dfJeu.NOMBRE.isnull()].index, inplace=True)
    assert all(dfJeu.NOMBRE == 1)
        
    # Effort
    dfJeu['EFFORT'] = len(passages)
        
    # Nettoyage
    dfJeu.drop(['PER5MN', 'PER10MN'], axis='columns', inplace=True)
    
    return dfJeu

In [ ]:
def ajouterAbsences(dfJeu, effort, pointsPapier):
    
    assert not dfJeu.empty, 'Erreur : Il n\'y aurait que des absences !'

    zone, surface, espece = dfJeu.iloc[0][['ZONE', 'HA', 'ESPECE']]
    dAbsence = { 'ZONE': zone, 'HA': surface, 'POINT': None, 'ESPECE': espece,
                 'DISTANCE': np.nan, 'EFFORT': effort, 'MALE': None,
                 'NOMBRE': np.nan, 'DATE': pd.NaT, 'OBSERVATEUR': None, 'PASSAGE': None }

    pointsManquants = [p for p in pointsPapier if p not in dfJeu.POINT.unique()]
    for p in pointsManquants:
        dAbsence.update(POINT=p)
        dfJeu = dfJeu.append(dAbsence, ignore_index=True)
    
    dfJeu.sort_values(by=['POINT'], inplace=True)

    return dfJeu, len(pointsManquants)

In [ ]:
# Paramètres généraux.
workDir = 'ACDC-Auto'
runEngine = False # Pas d'appel à l'exe si False, juste pour les fichiers d'entrée.

In [ ]:
# Tous les points effectués (pour absences).
pointsPapier = \
    list(map(int, """23,39,40,41,42,55,56,57,58,59,60,72,73,74,75,76,88,89,90,91,
                     105,106,109,110,112,113,122,123,125,126,127,128,129,130,141,142,143,144,145,146,
                     147,148,157,158,159,160,161,162,163,164,165,166,174,175,176,177,178,179,180,181,
                     182,183,184,185,192,193,194,195,196,197,198,199,200,201,202,210,211,212,213,214,
                     215,216,218,219,228,229,232,233,245,246,247,250,262,263,265,266,280,281,282,283,
                     284,299,300,301""".split(',')))

# Données brutes saisies par les observateurs, déjà individualisées, que les mâles.
ficDonnees = os.path.join(workDir, 'ACDC2019-Papyrus-DonneesBrutes.xlsx')
dfMales = pd.read_excel(ficDonnees, sheet_name='ResultIndivMales')
dfMales.rename(columns={ 'ha': 'HA', 'Distance en m': 'DISTANCE', 'Mâle\xa0?': 'MALE', 'Date': 'DATE',
                         'Période': 'PASSAGE', '0-5mn': 'PER5MN', '5-10 mn': 'PER10MN' }, inplace=True)

assert all(dfMales.MALE.str.lower() == 'oui')

print('Nb mâles   :', len(dfMales))
print('Nb espèces :', len(dfMales.ESPECE.unique()))

# Les espèces et passages à traiter.
dfToDo = pd.read_excel(ficDonnees, sheet_name='AFaire')
toDoCols = ['ESPECE', 'MALES', 'PERIODE']
assert all(col in dfToDo.columns for col in toDoCols)
dfToDo = dfToDo.reindex(toDoCols, axis='columns')
dfToDo.sort_values(by='MALES', ascending=False, inplace=True)

print('Espèces à traiter :', len(dfToDo))

# Les paramètres de toutes analyses à faire à chaque fois.
dfParams = pd.read_excel(ficDonnees, sheet_name='ParamsAnalyses')
paramCols = ['KeyFn', 'AdjustFn', 'Criterion', 'CVInterval']
assert all(col in paramCols for col in dfParams.columns)
dfParams = dfParams.reindex(paramCols, axis='columns')

print('Variantes d\'analyses :', len(dfParams))

In [ ]:
_ = implib.reload(ads)

In [ ]:
# Hack up normal installation root folders, in order to test without really installing Distance
ads.DSEngine.DistancePossInstPaths = ['AutoDS']

In [ ]:
# Le moteur
mcds = ads.MCDSEngine(workDir=workDir,
                         distanceUnit='Meter', areaUnit='Hectare',
                         surveyType='Point', distanceType='Radial')

# Les résultats de chaque analyse
dfResultats = None

#Pour chaque espèce à traiter
for index, sToDo in dfToDo.iterrows():

    espece, nbIndivs, passage = sToDo
    passages = [p for p in passage]

    print(espece, ':', passage)

    # Pour les 2 durées d'inventaire (sur chaque point)
    for duree in ['5mn', '10mn']:

        print('-', duree, end=' : ')

        # Sélection des données
        dfJeu = extraireJeuDonnees(dfMales, espece, passages, duree)
        print(len(dfJeu), 'mâles', end=', ')

        # Ajout des lignes d'absence
        dfJeu, nAbsences = ajouterAbsences(dfJeu, effort=len(passages), pointsPapier=pointsPapier)
        print(nAbsences, 'absences', end=' ')

        # Pour chaque précision numérique sur la distance (en décroissant)
        for precDist in [None, 1]:
            
            # Arrondi à la précision.
            if precDist is not None:
                dfJeu.DISTANCE = dfJeu.DISTANCE.apply(round, ndigits=precDist)
            
            # Voici donc le jeu de données
            jeu = ads.DataSet(dfJeu, decimalFields=['EFFORT', 'DISTANCE', 'NOMBRE'])
            
            # Pour chaque jeu de paramètres d'analyse
            for index, sParams in dfParams.iterrows():

                precision = 'tt' if precDist is None else precDist
                prfxAnalyse = '{}-{}-{}-{}dec-{}'.format(espece, duree, passage, precision, index)
                analyse = ads.MCDSAnalysis(engine=mcds, dataSet=jeu, namePrefix=prfxAnalyse,
                                           estimKeyFn=sParams['KeyFn'], estimAdjustFn=sParams['AdjustFn'],
                                           estimCriterion=sParams['Criterion'], cvInterval=sParams['CVInterval'])

                dResultat = odict([('espece', espece), ('passages', passage), ('duree', duree),
                                   ('precision', precision), ('index', index)])
                dResultat.update(analyse.run(realRun=runEngine))

                if dfResultats is None:
                    dfResultats = pd.DataFrame(columns=dResultat.keys())
                dfResultats = dfResultats.append(dResultat, ignore_index=True)
                
                #raise StopIteration()

# Sauvegarde des résultats
dfResultats.to_excel(workDir+'/ACDC2019-Papyrus-ResultatsAnalyses.xlsx')

In [ ]:
dResultat

In [ ]:
dfResultats